# Advanced Data Manipulation Language

## **Getting Started**

This week we will be learning more about Data Manipulation Language, as well as some SQL key words that will make our scripts more usable and secure.

## **Transactions**

When scripting DML operations we can use _transactions_ to to ensure that DML operations are executed as a single unit of work. Transactions follow the ACID properties:

- Atomicity- Ensures that all operations within a transaction are copmleted succesfully. If any operation fails, the entire transaction is rolled back, and the database remains unchanged.
- Consistency- Ensures that a transaction brings the database from one valid state to another valid state. The database must remain in a consistent state before and after the transaction.
- Isolation- Ensures that the operations of a transaction are isolated from the other transactions. Transactions are executed in sucha. way that they appear to be running in isolation from one another.
- Durability- Ensures that once a transaction has been committed, it will remain so, even in the event of a system failure. The changes made by the transaction are permanently recorded in the database.

To start a transcation, we use the keyword BEGIN. When ending a transcation we use either COMMIT or ROLLBACK depending on if the operation was succesful. In the below cell we are adding a record to the airline table, and then adding a flight for that airline to the flight table.

In [ ]:
BEGIN TRANSACTION;

--Insert row into airline
INSERT INTO ap_airline VALUES (8,'Super-Air-ior');

--Insert row in flights
INSERT INTO ap_flight VALUES (9,8,'SU872','Seattle');

COMMIT;

In [ ]:
--View both tables to see the new data
SELECT * FROM ap_airline
SELECT * FROM ap_flight

We can see that both queries were executed succesfully. Please note the below differences when writing transactions compared to the SQL we have been using so far:

- Each line of code ends with a semi-colon ;
- Each line is commented to describe it's purpose
- We start with BEGIN TRANSACTION and end with COMMIT

While this may not seem that different than what we have done in the past, writing our queries in this way makes them more _secure._ The below query is written to generate an error. We are going to perform the same two operations that we did before, however, this time we will be inserting a record in the flight table an incorrect data type.

In [ ]:
BEGIN TRANSACTION;

--Insert row into airline
INSERT INTO ap_airline VALUES (9,'Air Air Air');

--Insert row in flights
INSERT INTO ap_flight VALUES (10,'k','AA345','Denver'); --'k' is an invalid value for airline_id

COMMIT;

In [ ]:
--View both tables to see what happened
SELECT * FROM ap_airline
SELECT * FROM ap_flight

After viewing both tables you may notice that even though the record for ap\_airline was valid, and the output said that 1 row was affected, the record is not present. This is because the changes to the ap\_airline table were rolled back when a later query in the transaction failed. This is a security measure so that processes are not able to be half-executed. The transaction either completes succesfully or is cancelled.

While the syntax for writing transactions is a little different than what we have donoe previously, the syntax of our DML queries stay the same. We are just wrapping them into a container that ensures proper execution.

## **Error Handling**

SQL has error handling tools similar to other programming languages that you may have used before. Transactions are one type of error handling. Another error handling tool we will be learning about is TRY/CATCH. The TRY keyword denotes script that the system will _try_ to run. If there is a problem, the CATCH keyword denotes the script that will be ran once an error occurs. TRY/CATCH can be a powerful tool for building SQL scripts that have clear outputs and secure data manipulation. The below cell implements a TRY/CATCH with a DML transcation that is built to fail:

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        --Insert a new record into passport
        INSERT INTO ap_passport VALUES (6,'P87654',10); --insert a passport record for customer_id 10. There is no customer_id 10 in the customer table
        --Commit the transaction if there are no errors
        COMMIT;
    END TRY
    BEGIN CATCH
        --Roll back the transaction if an error occurs
        ROLLBACK
        PRINT 'An error occured, the transaction was rolled back.'
    END CATCH

Here is a breakdown of how the above cell executes:

1. Transaction begins
2. They system attempts to run the INSERT statement
3. The statement fails because it is trying to insert a value for a FK that does not exist in the referenced table (customer)
4. Due to the failure, the cocde execution is deferred to the CATCH block
5. The command to rollback the transaction is cancelled
6. The PRINT keyword gives feedback to the user about what happened

The indetion of each block is not required, but is best practice and should be used to assist with readability. In the below cell is a similar operation where the transaction can execute without errors:

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        --Insert new record into customer
        INSERT INTO ap_customer VALUES (10,'Mike','Willstone','2000-12-10');
        --Insert new record into passport
        INSERT INTO ap_passport VALUES (6,'P3732',10);
        --Insert new record into boarding pass:
        INSERT INTO ap_boarding_pass VALUES (5,10,1,'BP45678906545678');
        --Output confirmation of success
        PRINT 'Sucessfully inserted data into customer, passport, and flight'
        --Commit data to database
        COMMIT;
    END TRY
    BEGIN CATCH
        --Output error message
        PRINT 'Error Message: ' + ERROR_MESSAGE();
        --Rollback transactions on error
        ROLLBACK;
    END CATCH

Upon running the above cell, you are returned the message indicating successful insertion of our data.

  

Run the above cell again. You will notice that the message from the CATCH block containing the error is output instead. We can include the error that caused the CATCH to be triggered by using the ERROR\_MESSAGE() function, like above.

## **JOINS and Subqueries in DML**

Sometimes we will need information from another table to perform our DML operations. In these instances we can use JOINs and subqueries to make our DML scripts more powerful. In the below example we will join boarding\_pass to customer to delete a boarding pass for a specific customer using their name.

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        --DELETE record from boarding_pass table based on customer name
        DELETE bp
        FROM ap_boarding_pass bp --DELETE queries don't allow AS for aliases, so you just put the alias after the table without AS
        JOIN ap_customer c ON bp.customer_id = c.customer_id
        WHERE c.first_name = 'Michael' AND c.last_name = 'Brown';
        --Commit change
        COMMIT;
    END TRY
    BEGIN CATCH
        --Output error message
        PRINT 'Error Message ' + ERROR_MESSAGE();
        --Rollback transaction
        ROLLBACK;
    END CATCH;

While we could have accomplished the above query by specifying the customer\_id and not using a JOIN, we won't always know information like the id, and users almost never will. Building scripts like this makes them more user friendly and easier to work with. Below is an example where we do the same thing using a subquery instead of a JOIN. Zero rows will be affected because we already deleted this customer in the previous query. Note that this does not resolve in an error, there is just no action taken.

In [ ]:
BEGIN TRANSACTION;
    BEGIN TRY
        --DELETE record from boarding_pass table based on customer name
        DELETE bp
        FROM ap_boarding_pass bp
        WHERE bp.customer_id IN (
            --Select the customer_id that matches the name we are searching for
            SELECT c.customer_id
            FROM ap_customer c
            WHERE c.first_name = 'Michael' AND c.last_name = 'Brown'
        );
        --Commit change
        COMMIT;
    END TRY
    BEGIN CATCH
        --Output error message
        PRINT 'Error Message ' + ERROR_MESSAGE();
        --Rollback transaction
        ROLLBACK;
    END CATCH;

You may have noticed that not every singl eline ends with a semicolon. The only times that we use a semicolon is at the end of a complete statement. For example, a query like:

> _UPDATE ap\_airline SET airline\_name = 'Super Duper Air' WHERE airline\_id = 3_

is a complete statement, so we would use a semi colon. We do not put a semicolon after something like BEGIN TRY because that is just the beginning of a statement. The end of that statement is END CATCH so we would put the semicolon after that. Examine the above query to better understand these patterns.

## **Exercises**

For these exercises we will be using the Storage Unity database. If you have not already, I would strongly recommend having access to the ERD linked in Github for these exercises.

1\. In the code cell below, write a transaction that:

- Adds a new record to the unit table
    - unit\_id- 71
    - unit\_number- A71
    - type\_id- 3
    - facility\_id- 2
    - status- occupied
- Adds a new record to the rental table
    - rental\_id- 154
    - unit\_id- 71
    - customer\_id- 10
    - start\_date- 5/14/2024
    - end\_date- 5/13/2025
    - payment\_option- Y
    - discount\_amount- 0

In [2]:
--Your code here
BEGIN TRANSACTION;

INSERT INTO su_unit (unit_id, unit_number, type_id, facility_id, status)
VALUES (71, 'A71', 3, 2, 'occupied');

INSERT INTO su_rental (rental_id, unit_id, customer_id, start_date, end_date, payment_option, discount_amount)
VALUES (154, 71, 10, '2024-05-14', '2025-05-13', 'Y', 0);

COMMIT TRANSACTION;




(1 row affected)

(1 row affected)

Total execution time: 00:00:00.110

2\. In the below code cell, write a query that returns the price per year of the unit that was rented out in exercise 1. (No DML required, result should return a single value)

In [3]:
--Your code here
SELECT t.price_per_year
FROM su_type t
JOIN su_unit u ON t.type_id = u.type_id
WHERE u.unit_id = 71;


(1 row affected)

Total execution time: 00:00:00.088

price_per_year
580.00


3\. In the below code cell, write a transaction that:

- Adds a record to the invoice table for the rental added in exercise 1
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">invoice_id- 201</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">rental_id- 154</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">amount- Use the query written in exercise 2 as a subquery to get this value</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">invoice_date- 5/13/2025</span>
- <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Updates the status of unit 71 to be maintenance</span>

In [ ]:
--Your code here
BEGIN TRANSACTION;

INSERT INTO su_invoice (invoice_id, rental_id, amount, invoice_date)
VALUES (201, 154, (SELECT t.price_per_year
                   FROM su_type t
                   JOIN su_unit u ON t.type_id = u.type_id
                   WHERE u.unit_id = 71), '2025-05-13');

UPDATE su_unit
SET status = 'maintenance'
WHERE unit_id = 71;

COMMIT TRANSACTION;


4\. In the below code cell, write a transaction that:

- Adds a record to the payment table
    - payment\_id- Use a subquery to find the MAX of payment\_id _and_ add 1.
    - invoice\_id- 201
    - amount- Create a subquery that pulls the amount of the invoice for invoice #201
    - payment\_date- 5/15/2025
    - payment\_method- cash
- Update the status of unit 71 to be available

In [5]:
--Your code here
BEGIN TRANSACTION;

INSERT INTO su_payment (payment_id, invoice_id, amount, payment_date, payment_method)
VALUES (
    (SELECT MAX(payment_id) + 1 FROM su_payment),
    201, 
    (SELECT amount FROM su_invoice WHERE invoice_id = 201), 
    '2025-05-15', 
    'cash'
);

UPDATE su_unit
SET status = 'available'
WHERE unit_id = 71; 

COMMIT TRANSACTION;


: Msg 547, Level 16, State 0, Line 4
The INSERT statement conflicted with the FOREIGN KEY constraint "FK__su_paymen__invoi__17F790F9". The conflict occurred in database "MS0885011", table "dbo.su_invoice", column 'invoice_id'.

The statement has been terminated.

(1 row affected)

Total execution time: 00:00:01.009

## **Scenario**

The storage unit facility needs a SQL script that can do everything that we did above. The challenege will be to use subqueries to implement stronger automation. They way that our database works, a new record is always inserted with the highest ID for that table. Create a single transaction that accomplishes the following:

- Add a record to the unit table
    - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">unit_id- (Max of unit_id + 1)</span>
    - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">unit_number- (Concatenate 'A' and (Max of unit_id +1))</span>
        - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">You may have to do a little research for this. Look into using CAST/CONVERT.</span>
        - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">I would try getting this to work in a simple SELECT query before trying it in your transaction</span>
    - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">type_id- 4</span>
    - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">facility_id- 3</span>
    - <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">status- occupied</span>
- Adds a new record to the rental table
    - rental\_id- (Max of rental\_id + 1)
    - unit\_id- (Max of unit\_id (do not add 1))
    - customer\_id- 15
    - start\_date- 6/14/2024
    - end\_date- 6/13/2025
    - payment\_option- Y
    - discount\_amount- 30
- Adds a record to the invoice table for the new rental
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">invoice_id- (Max of invoice_id + 1)</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">rental_id- (Max of rental_id (Do not add 1))<br></span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">amount- Use a subquery to pull the appropriate price for the unit and subtract the discount amount of the rental</span>
    - invoice\_date- 6/13/2025
- <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">Updates the status of the unit to be maintenance</span>
    - <span style="color: var(--vscode-foreground); font-family: -apple-system, BlinkMacSystemFont, sans-serif;">unit id of the unit that needs updating will be Max of unit_id</span>
- Adds a record to the payment table
    - payment\_id- payment\_id + 1
    - invoice\_id- (Max of invoice\_id (Do not add 1))
    - amount- Create a subquery that pulls the amount of the invoice where invoice\_id is the Max of invoice\_id
    - payment\_date- 6/15/2025
    - payment\_method- debit
- Update the status of the unit to be available

This is a challenging transaction to write! I would recommend programing each individual DML query and then put them into a transaction style script. Before testing, make sure that you are using TRY/CATCH and implementing a ROLLBACK in your catch block so that changes don't stick if something is wrong.

  

If you need your database reset, please reach out! Also, don't forget that you can seek assistance in the Homework Help channel in Teams.

In [6]:
--Your code here
BEGIN TRANSACTION;

INSERT INTO su_unit (unit_id, unit_number, type_id, facility_id, status)
VALUES (
    (SELECT MAX(unit_id) + 1 FROM su_unit),
    CONCAT('A', CAST((SELECT MAX(unit_id) + 1 FROM su_unit) AS VARCHAR(3))), 
    4,
    3,
    'occupied'
);

INSERT INTO su_rental (rental_id, unit_id, customer_id, start_date, end_date, payment_option, discount_amount)
VALUES (
    (SELECT MAX(rental_id) + 1 FROM su_rental),
    (SELECT MAX(unit_id) FROM su_unit),
    15,
    '2024-06-14', 
    '2025-06-13',
    'Y', 
    30 
);

INSERT INTO su_invoice (invoice_id, rental_id, amount, invoice_date)
VALUES (
    (SELECT MAX(invoice_id) + 1 FROM su_invoice),
    (SELECT MAX(rental_id) FROM su_rental), 
    (SELECT t.price_per_year - r.discount_amount
     FROM su_type t
     JOIN su_unit u ON t.type_id = u.type_id
     JOIN su_rental r ON u.unit_id = r.unit_id
     WHERE u.unit_id = (SELECT MAX(unit_id) FROM su_unit)),
    '2025-06-13'
);

UPDATE su_unit
SET status = 'maintenance'
WHERE unit_id = (SELECT MAX(unit_id) FROM su_unit);

INSERT INTO su_payment (payment_id, invoice_id, amount, payment_date, payment_method)
VALUES (
    (SELECT MAX(payment_id) + 1 FROM su_payment),
    (SELECT MAX(invoice_id) FROM su_invoice),
    (SELECT amount FROM su_invoice WHERE invoice_id = (SELECT MAX(invoice_id) FROM su_invoice)),
    '2025-06-15',
    'debit'
);

UPDATE su_unit
SET status = 'available'
WHERE unit_id = (SELECT MAX(unit_id) FROM su_unit);

COMMIT TRANSACTION;


(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:02.261